In [1]:
import pandas as pd
import numpy as np
import peakutils
import scipy.fftpack
import matplotlib.pyplot as plt
#%matplotlib inline
#import mpld3
#mpld3.enable_notebook()
#import glob
#print(glob.glob("txt/*.txt"))
from os import listdir
from os.path import isfile, join
files = [f for f in listdir("txt") if isfile(join("txt", f))]

In [2]:
def fft_fig_out(filename,peak_thres):
    print('Generate graph for '+filename)
    data = pd.read_csv('txt/'+filename, sep="\t", header=None)
    #len_input = int(input("How many points you want to plot"))
    len_input = 30000
    data_2=data.iloc[2:len_input+2,[1,2,3]].copy(deep=False).reset_index(drop=True).apply(pd.to_numeric)
    #By is object
    data_2.columns = ["Bx", "By", "Bz"]
    #print(data_2.head())
    data2_len = len(data_2)
    #print('\nThis test has points of:'+ str(data2_len) )
    x_values =  np.linspace(0,data2_len*0.00002,data2_len,endpoint=False)
    fig, axarr = plt.subplots(3,2, sharex='col',figsize=(12,10))
    #fig.tight_layout()
    #fig.tight_layout(pad=1.5, h_pad=1.2, w_pad=1.2, rect=None)
    plt.subplots_adjust(hspace = 0.3,wspace = 0.4)
    axarr[0,0].scatter(x_values,data_2['Bx'],s=10, c='red')
    axarr[0,0].set(ylabel='Magnetic Field (mT)',xlabel='Time (s)')
    axarr[0,0].legend(loc='best')
    axarr[1,0].scatter(x_values,data_2['By'],s=10, c='red')
    axarr[1,0].set(ylabel='Magnetic Field (mT)',xlabel='Time (s)')
    axarr[1,0].legend(loc='best')
    axarr[2,0].scatter(x_values,data_2['Bz'],s=10, c='red')
    axarr[2,0].set(ylabel='Magnetic Field (mT)',xlabel='Time (s)')
    axarr[2,0].legend(loc='best')
    f = np.linspace(0.0, 1.0/(2.0*1/50000), data2_len/2)
    

    Bxfft=2.0/data2_len * np.abs(scipy.fftpack.fft(data_2['Bx'])[:data2_len//2])
    axarr[0,1].plot(f, Bxfft,'rs-')
    axarr[0,1].set_xlim(0,360)
    axarr[0,1].xaxis.set(ticks=[0,60,120,180,240,300,360],ticklabels=[0,60,120,180,240,300,360])
    axarr[0,1].set(ylabel='Magnetic Field (mT)',xlabel='Frequency (Hz)')
    axarr[0,1].set_title('Bx FFT')
    axarr[0,1].grid()
    ymax_Bxfft = max(Bxfft)
    Bxfft_peaks_indices=peakutils.indexes(Bxfft, thres=peak_thres, min_dist=10)[0:2]
    #print(Bxfft_peaks_indices)
    Bxfft_peaks_indices=[36,108] # for len_input = 30000
    #print(f[Bxfft_peaks_indices])
    #print(Bxfft[Bxfft_peaks_indices])
    xpos_Bxfft = Bxfft.tolist().index(ymax_Bxfft)
    #xpos_Bxfft = Bxfft.argmax(ymax_Bxfft)
    xmax_Bxfft = f[xpos_Bxfft]
    #axarr[0,1].text(220, 0.02,str(format(xmax_Bxfft,'.3g'))+'Hz Bx:'+str(format(ymax_Bxfft,'.3g'))+'mT',verticalalignment='bottom', 
    #            horizontalalignment='right',)
            


    Byfft=2.0/data2_len * np.abs(scipy.fftpack.fft(data_2['By'])[:data2_len//2])
    axarr[1,1].plot(f, Byfft,'go-')
    axarr[1,1].set(ylabel='Magnetic Field (mT)',xlabel='Frequency (Hz)')
    axarr[1,1].set_title('By FFT')
    axarr[1,1].grid()
    Byfft_peaks_indices=peakutils.indexes(Byfft, thres=peak_thres, min_dist=10)[0:2]
    #print(Byfft_peaks_indices)
    #print(f[Byfft_peaks_indices])
    #print(Byfft[Byfft_peaks_indices])
    ymax_Byfft = max(Byfft)
    xpos_Byfft = Byfft.tolist().index(ymax_Byfft)
    xmax_Byfft = f[xpos_Byfft]
    #axarr[1,1].text(220, 0.8,str(format(xmax_Byfft,'.3g'))+'Hz By:'+str(format(ymax_Byfft,'.3g'))+'mT',verticalalignment='bottom', 
    #            horizontalalignment='right',)
   


    Bzfft=2.0/data2_len * np.abs(scipy.fftpack.fft(data_2['Bz'])[:data2_len//2])
    axarr[2,1].plot(f,Bzfft,'bp-')
    axarr[2,1].set_title('Bz FFT')
    axarr[2,1].set(ylabel='Magnetic Field (mT)',xlabel='Frequency (Hz)')
    axarr[2,1].grid()
    Bzfft_peaks_indices=peakutils.indexes(Bzfft, thres=peak_thres, min_dist=10)[0:2]
    #print(Bzfft_peaks_indices)
    #use Bx indices with same frequency
    #print(f[Bxfft_peaks_indices])
    #print(Bzfft[Bxfft_peaks_indices])
    #print(Bzfft[Bxfft_peaks_indices][1])
    #print(str(format(f[Bxfft_peaks_indices][1],'.3g'))+' Hz:'+str(format(Bzfft[Bxfft_peaks_indices][1],'.3g'))+'mT')
    ymax_Bzfft = max(Bzfft)
    xpos_Bzfft = Bzfft.tolist().index(ymax_Bzfft)
    xmax_Bzfft = f[xpos_Bzfft]
    
    #axarr[2,1].text(220, 0.015,str(format(xmax_Bzfft,'.3g'))+'Hz By:'+str(format(ymax_Bzfft,'.3g'))+'mT',verticalalignment='bottom', 
    #            horizontalalignment='right',)

    fig.suptitle('Bx '+str(format(f[Bxfft_peaks_indices][0],'.3g'))+' Hz:'+str(format(Bxfft[Bxfft_peaks_indices][0],'.3g'))+'mT    '
                 +str(format(f[Bxfft_peaks_indices][1],'.3g'))+' Hz:'+str(format(Bxfft[Bxfft_peaks_indices][1],'.3g'))+'mT'
                 +'\n'
                 +'By '+str(format(f[Bxfft_peaks_indices][0],'.3g'))+' Hz:'+str(format(Byfft[Bxfft_peaks_indices][0],'.3g'))+'mT    '
                 +str(format(f[Bxfft_peaks_indices][1],'.3g'))+' Hz:'+str(format(Byfft[Bxfft_peaks_indices][1],'.3g'))+'mT'
                 +'\n'
                 'Bz '+str(format(f[Bxfft_peaks_indices][0],'.3g'))+' Hz:'+str(format(Bzfft[Bxfft_peaks_indices][0],'.3g'))+'mT    '
                 +str(format(f[Bxfft_peaks_indices][1],'.3g'))+' Hz:'+str(format(Bzfft[Bxfft_peaks_indices][1],'.3g'))+'mT',
                 fontsize=14, fontweight='bold')

    fig.savefig('figure/'+filename+'.png')
    
    result_df=pd.DataFrame({"180Hz (mT)":[format(Bxfft[Bxfft_peaks_indices][1],'.3g'),format(Byfft[Byfft_peaks_indices][1],'.3g'),format(Bzfft[Bxfft_peaks_indices][1],'.3g')],
                           "60Hz (mT)":[format(Bxfft[Bxfft_peaks_indices][0],'.3g'),format(Byfft[Byfft_peaks_indices][0],'.3g'),format(Bzfft[Bxfft_peaks_indices][0],'.3g')]},
                           index=[filename[0:6]+'Bx',filename[0:6]+'By',filename[0:6]+'Bz'])
    #print(result_df)
    return result_df

In [3]:
fft_fig_out('30 mm AC Measurements_1-12-2018_11-14-40 PM.txt',0.1)
#y 12 mm AC Measurements_1-12-2018_11-39-41 PM
#0_AC Measurements_1-12-2018_10-35-59 PM
#24 mm AC Measurements_1-12-2018_11-13-16 PM not working

Generate graph for 30 mm AC Measurements_1-12-2018_11-14-40 PM.txt


,180Hz (mT),60Hz (mT)
30 mm Bx,0.078,0.223
30 mm By,0.0209,0.0543
30 mm Bz,0.022,0.0584


In [4]:
df_empty = pd.DataFrame({'180Hz (mT)' : [],'60Hz (mT)' : []})
df_combine=pd.concat([df_empty,fft_fig_out('24 mm AC Measurements_1-12-2018_11-13-16 PM.txt',0.1)])
print(df_combine)

Generate graph for 24 mm AC Measurements_1-12-2018_11-13-16 PM.txt
         180Hz (mT) 60Hz (mT)
24 mm Bx     0.0781     0.228
24 mm By     0.0224    0.0608
24 mm Bz     0.0191    0.0403


In [5]:
df_combine = pd.DataFrame({'180Hz (mT)' : [],'60Hz (mT)' : []})
for file in files:
    df_combine=pd.concat([df_combine,fft_fig_out(file,0.1)])
    
df_combine.to_csv('all_harmonics.csv', sep=',', encoding='utf-8', index=True)

Generate graph for 0_AC Measurements_1-12-2018_10-35-59 PM.txt
Generate graph for 10 mm AC Measurements_1-12-2018_10-40-49 PM.txt
Generate graph for 12mm AC Measurements_1-12-2018_10-47-54 PM.txt
Generate graph for 14 mm AC Measurements_1-12-2018_10-48-44 PM.txt
Generate graph for 16 mm AC Measurements_1-12-2018_10-49-27 PM.txt
Generate graph for 18 mm AC Measurements_1-12-2018_10-49-55 PM.txt
Generate graph for 20 mm AC Measurements_1-12-2018_11-12-22 PM.txt
Generate graph for 22 mm AC Measurements_1-12-2018_11-12-51 PM.txt
Generate graph for 24 mm AC Measurements_1-12-2018_11-13-16 PM.txt
Generate graph for 26 mm AC Measurements_1-12-2018_11-13-35 PM.txt
Generate graph for 28 mm AC Measurements_1-12-2018_11-14-13 PM.txt
Generate graph for 2mm_AC Measurements_1-12-2018_10-37-19 PM.txt
Generate graph for 30 mm AC Measurements_1-12-2018_11-14-40 PM.txt
Generate graph for 32 mm AC Measurements_1-12-2018_11-15-02 PM.txt
Generate graph for 34 mm AC Measurements_1-12-2018_11-15-19 PM.txt
Ge

C:\Users\zxu\AppData\Local\Continuum\Anaconda3\lib\site-packages\matplotlib\pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Generate graph for 44 mm AC Measurements_1-12-2018_11-17-12 PM.txt
Generate graph for 46 mm AC Measurements_1-12-2018_11-17-45 PM.txt
Generate graph for 48 mm AC Measurements_1-12-2018_11-18-04 PM.txt
Generate graph for 4mm_AC Measurements_1-12-2018_10-37-53 PM.txt
Generate graph for 50 mm AC Measurements_1-12-2018_11-18-28 PM.txt
Generate graph for 52 mm AC Measurements_1-12-2018_11-28-23 PM.txt
Generate graph for 54 mm AC Measurements_1-12-2018_11-28-51 PM.txt
Generate graph for 6mm_AC Measurements_1-12-2018_10-39-52 PM.txt
Generate graph for 8mm AC Measurements_1-12-2018_10-40-22 PM.txt
Generate graph for y 10mm AC Measurements_1-12-2018_11-38-23 PM.txt
Generate graph for y 12 mm AC Measurements_1-12-2018_11-39-41 PM.txt
Generate graph for y 2mm AC Measurements_1-12-2018_11-35-56 PM.txt
Generate graph for y 4mm AC Measurements_1-12-2018_11-36-42 PM.txt
Generate graph for y 6mm AC Measurements_1-12-2018_11-37-17 PM.txt
Generate graph for y 8mm AC Measurements_1-12-2018_11-37-45 PM.tx